# Example querying and joining various consdb tables and also EFD data not in consdb # 

In [ ]:
import os
import logging

logging.getLogger("rubin_nights").setLevel(logging.INFO)

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import colorcet as cc
from IPython.display import display, HTML

from astropy.time import Time, TimeDelta

from rubin_nights import connections

import rubin_nights.dayobs_utils as rn_dayobs
from rubin_nights import observatory_status

In [ ]:
# set up connections to consdb, efd, etc.
# note that `get_clients` will do some sleuthing to try to find your relevant site (but you can override the value if needed)
# it will also try to find your RSP token if you are running outside of an RSP and thus need it. 
# See more in the docstring info at https://github.com/lsst-sims/rubin_nights/blob/main/rubin_nights/connections.py

endpoints = connections.get_clients()

print(endpoints)

In [ ]:
# Are you looking for information over a given time period? Maybe an entire dayobs? 

day_obs = 20260110

# Set start and end of period of interest .. example for entire dayobs
t_start = rn_dayobs.day_obs_to_time(day_obs)
t_end = t_start + TimeDelta(1, format='jd')

In [ ]:
# Get visit1 + visit1_quicklook data
# use consdb or consdb_tap endpoints .. either is ok
consdb = endpoints['consdb_tap']

# This is just all visits
visits = consdb.get_visits("lsstcam", t_start, t_end)
len(visits), visits.seq_num.min(), visits.seq_num.max()

In [ ]:
visits.query("img_type in ('science')")[['visit_id', 'img_type',  'target_name', 'observation_reason', 'science_program', 'zero_point_median', 'psf_sigma_median']].head(3)

In [ ]:
# Or send a specific query, including querying other tables, etc.
# Schema online at https://sdm-schemas.lsst.io

# You CAN join on ccdvisit here, but if your primary interest is a per-visit number, 
# it can be simpler to query the per-detector values separately, consolidate what you need to per-visit via a "groupby" 
# and then join the result to the per-visit values.
# Otherwise, you inflate all of your per-visit values to be per (189) detectors. 

# lazy way to get some columns .. could list them yourself though
# This particular query only works through RestAPI (i.e. endpoints['consdb'])
query = "select column_name from information_schema.columns where table_name = 'visit1_quicklook' and table_schema = 'cdb_lsstcam'"
tt = endpoints['consdb'].query(query)
quicklook_cols = [c for c in tt.column_name.values 
            if (c.startswith('z') and not c.startswith('zero')) 
            or 'aos' in c or 'donut' in c or 'ringss' in c or 'dimm' in c 
            or 'guider' in c or 'psf_sigma_median' in c]

query = "select column_name from information_schema.columns where table_name = 'visit1_efd' and table_schema = 'efd_lsstcam'"
tt = endpoints['consdb'].query(query)
tefd_cols = [c for c in tt.column_name.values if 'dimm' in c]
tefd_cols

# build the query with these columns 
query = "select v.*, "
query += ", ".join([f"q.{col}" for col in quicklook_cols])
query += ", "
query += ", ".join([f"efd.{col}" for col in tefd_cols])
query += ( 
    " from cdb_lsstcam.visit1 as v "\
    "left join cdb_lsstcam.visit1_quicklook as q "\
    "on v.visit_id = q.visit_id "\
    "left join efd_lsstcam.visit1_efd as efd "\
    "on v.visit_id = efd.visit_id ")
# Add what constraints you need
query += f" where v.day_obs = {day_obs} and v.science_program = 'BLOCK-419'"

visit_data = consdb.query(query)
platescale = 0.2
sigma_to_fwhm = 2.0 * np.sqrt(2.0 * np.log(2.0))
visit_data['fwhm_median'] = visit_data['psf_sigma_median'] * platescale * sigma_to_fwhm

print(len(visit_data))
visit_data.head(3)[['science_program', 'observation_reason', 'visit_id', 'band', 'aos_fwhm', 'dimm_seeing', 'donut_blur_fwhm', 'fwhm_median', 'guider_psf_fwhm', 'guider_total_seeing']]

In [ ]:
# an example query that includes the ccdvisit1_quicklook tables directly into the join/query

# query = "select v.*, "
# query += "c.detector, "
# query += ", ".join([f"q.{col}" for col in quicklook_cols])
# query += ", "
# query += ", ".join([f"ccdq.{col}" for col in ccdquicklook_cols])
# query += ", "
# query += ", ".join([f"efd.{col}" for col in tefd_cols])
# query += ( 
#     " from cdb_lsstcam.visit1 as v "\
#     "join cdb_lsstcam.ccdvisit1 as c "\
#     "on v.visit_id = c.visit_id "\
#     "left join cdb_lsstcam.ccdvisit1_quicklook as ccdq "\
#     "on c.ccdvisit_id = ccdq.ccdvisit_id "\
#     "left join cdb_lsstcam.visit1_quicklook as q "\
#     "on v.visit_id = q.visit_id "\
#     "left join efd_lsstcam.visit1_efd as efd "\
#     "on v.visit_id = efd.visit_id ")
# query += f" where v.day_obs = {day_obs}"

In [ ]:
# Alternatively, query the ccdvisit1_quicklook (or other per-detector table), calculate the per-visit value and merge back with visit info above

# query - 
ccdvisit_quicklook_cols = ['n_psf_star', 'psf_sigma', 'psf_area', 'psf_ixx', 'psf_iyy', 'psf_ixy']

query = "select v.*, c.detector, "
query += ", ".join([f"cq.{col}" for col in ccdvisit_quicklook_cols])
query += ( 
         " from cdb_lsstcam.visit1 as v join cdb_lsstcam.ccdvisit1 as c on v.visit_id = c.visit_id "\
         "left join cdb_lsstcam.ccdvisit1_quicklook as cq on c.ccdvisit_id = cq.ccdvisit_id "\
         f"where c.detector < 189 and v.day_obs = {day_obs} and v.science_program = 'BLOCK-419'")
ccdvisit = consdb.query(query)

len(visit_data), len(ccdvisit), len(ccdvisit)/189

In [ ]:
# calculate per visit

# First add psf_e1 / psf_e2 / psf_e to each detector 
def calc_eccentricities(dd):
    denom = dd.psf_ixx + dd.psf_iyy
    psf_e1 = (dd.psf_ixx - dd.psf_iyy) / denom
    psf_e2 = (2 * dd.psf_ixy) / denom
    psf_e = np.sqrt(psf_e1**2 + psf_e2**2)
    new_df = pd.DataFrame([psf_e1, psf_e2, psf_e], index=['psf_e1', 'psf_e2', 'psf_e'], columns=dd.index).T
    return pd.merge(dd, new_df, left_index=True, right_index=True)

ccdvisit = calc_eccentricities(ccdvisit)


def agg_over_detectors(g):
    # I think it's worthwhile to reject detectors with <10 psf stars
    # It's probably also worthwhile to reject the detectors in the vignette region
    n_psf_star = g.n_psf_star.fillna(0)
    good = np.where(n_psf_star > 10)[0]
    n_det = len(good)
    psf_5 = np.nanpercentile(g.psf_sigma.iloc[good], 5)
    psf_95 = np.nanpercentile(g.psf_sigma.iloc[good], 95)
    psf_50 = np.nanpercentile(g.psf_sigma.iloc[good], 50)
    psf_e1_med = np.nanmedian(g.psf_e1.iloc[good])
    psf_e2_med = np.nanmedian(g.psf_e2.iloc[good])
    psf_e_med = np.nanmedian(g.psf_e.iloc[good])
    return pd.Series({"n_det": n_det, "psf_5": psf_5, "psf_50": psf_50, "psf_95": psf_95, 
                      "psf_e1_med": psf_e1_med, "psf_e2_med": psf_e2_med, "psf_e_med": psf_e_med})

grouped_ccdvisit = ccdvisit.groupby("visit_id").apply(agg_over_detectors, include_groups=False)

In [ ]:
# Merge with per-visit data from earlier
vdata = pd.merge(visit_data, grouped_ccdvisit, left_on='visit_id', right_index=True, how='outer')
vdata = vdata.sort_values(by='visit_id')
vdata.head(3)[['visit_id', 'band', 'altitude', 'azimuth', 'airmass',
       'aos_fwhm', 'dimm_seeing', 'donut_blur_fwhm', 'fwhm_median', 
       'guider_psf_fwhm', 'guider_total_seeing', 
       'n_det', 'psf_5', 'psf_50', 'psf_95', 'psf_e_med']]

In [ ]:
# And let's add something from the EFD.

sunset, sunrise = rn_dayobs.day_obs_sunset_sunrise(day_obs)

# I first tried the query for the temperatures using a group by time 
# But it turns out, the fill value means that we do get NaNs in the result.

# query = ('SELECT mean(temperatureItem6) AS tma_truss_plus_xy, mean(temperatureItem7) AS tma_truss_minus_xy '\
#         'FROM "efd"."autogen"."lsst.sal.ESS.temperature" '\
#          f"WHERE salIndex = 122 AND (time >= '{sunset.utc.isot}Z' AND time <= '{sunrise.utc.isot}Z') GROUP BY time(5s) FILL(null)")
# truss_temp = endpoints['efd'].query(query)

# Also, the values themselves weren't that large to retrieve - the update is every 2 seconds or so, but it's only a few values
# So let's do it this way, in this case.
truss_temp = endpoints['efd'].select_time_series("lsst.sal.ESS.temperature", ['temperatureItem6', 'temperatureItem7'], sunset, sunrise, index=122)
truss_temp.rename({"temperatureItem6": "tma_truss_plus_xy", "temperatureItem7": "tma_truss_minus_xy"}, axis=1, inplace=True)

In [ ]:
# And now to fill these values into the per-visit table ... 
# We could try interpolating - let's try something super simple

from scipy.interpolate import UnivariateSpline

# Convert to TAI to match more easily with visit times
truss_times_mjd = Time(truss_temp.index.values, scale='utc').tai.mjd 
spline_plus_xy = UnivariateSpline(truss_times_mjd, truss_temp.tma_truss_plus_xy.values, s=0)
spline_minus_xy = UnivariateSpline(truss_times_mjd, truss_temp.tma_truss_minus_xy.values, s=0)

truss_fit_plus_xy = spline_plus_xy(vdata.exp_midpt_mjd)
truss_fit_minus_xy = spline_minus_xy(vdata.exp_midpt_mjd)

vdata['tma_truss_plus_xy'] = truss_fit_plus_xy
vdata['tma_truss_minus_xy'] = truss_fit_minus_xy

# plt.figure()
# plt.plot(truss_times_mjd, truss_temp.tma_truss_plus_xy.values, label='+XY data', color='gray')
# plt.plot(vdata.obs_start_mjd, truss_fit_plus_xy, label='+XY Fit', color='red', lw=1)
# plt.plot(truss_times_mjd, truss_temp.tma_truss_minus_xy.values, label='-XY data', color='gray')
# plt.plot(vdata.obs_start_mjd, truss_fit_minus_xy, label='-XY Fit', color='blue', lw=1)
# plt.legend()
# plt.show()

In [ ]:
# Or we could even just pick the values which overlap the visit and take the mean of those 
# The difference in these methods looks like noise, but the fit above is 10 times faster so let's do that.

# Convert to TAI to match more easily with visit times
truss_times_mjd = Time(truss_temp.index.values, scale='utc').tai.mjd 
truss_temp['mjd'] = truss_times_mjd

def add_truss_temp(x, truss_temp):
    window = np.where((truss_temp['mjd'] >= x.obs_start_mjd) & (truss_temp['mjd'] <= x.obs_end_mjd))
    plus_xy = truss_temp.iloc[window].tma_truss_plus_xy.mean()
    minus_xy = truss_temp.iloc[window].tma_truss_minus_xy.mean()
    return pd.Series({"tma_truss_plus_xy": plus_xy, "tma_truss_minus_xy": minus_xy})

new_df = vdata.apply(add_truss_temp, args=[truss_temp], axis=1)

# plt.figure()
# plt.plot(truss_times_mjd, truss_temp.tma_truss_plus_xy.values, label='+XY data', color='gray')
# plt.plot(vdata.obs_start_mjd, new_df.tma_truss_plus_xy, label='+XY Fit', color='red', lw=1)
# plt.plot(truss_times_mjd, truss_temp.tma_truss_minus_xy.values, label='-XY data', color='gray')
# plt.plot(vdata.obs_start_mjd, new_df.tma_truss_minus_xy, label='-XY Fit', color='blue', lw=1)
# plt.legend()
# plt.show()

In [ ]:
fig, ax1 = plt.subplots(figsize=(10, 5))
ax2 = ax1.twinx()

ax1.plot(Time(vdata.obs_start_mjd, format='mjd', scale='tai').utc.to_datetime(), vdata.psf_50, color='k', label='psf_sigma')
ax2.plot(truss_temp.index, truss_temp.tma_truss_plus_xy, label='plus_xy')
ax2.plot(Time(vdata.obs_start_mjd, format='mjd', scale='tai').utc.to_datetime(), vdata.tma_truss_plus_xy, color='k', linestyle=':', label='tma_plus_xy')
ax2.plot(truss_temp.index, truss_temp.tma_truss_minus_xy, label='minus_xy')
ax1.tick_params(axis='x', labelrotation=45)
ax1.set_xlabel("Time (UTC)")
ax1.set_ylabel("PSF SIGMA (pixels)")
ax2.set_ylabel("Truss Temperature")
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
_ = ax1.set_title(f"DayObs {day_obs}")